### Tensorflow_Graph_evaluations
Description : Studying tensorlfow graph structure

In [1]:
import os
from six.moves import xrange  # pylint: disable=redefined-builtin
import PIL.Image as Image
import random
import numpy as np
import cv2
import time
import math
import matplotlib.pyplot as plt
import tensorflow as tf
from keras import backend as K
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from skimage import io
from sklearn import metrics
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings('ignore')

# Custom scripts
import lscript as lsp
import modelling as md

Using TensorFlow backend.


In [2]:
height=112 
width=112 
channel=3
crop_size=112

cluster_length=16
feature_size=8192
nb_classes=2
saved_path='/nobackup/leopauly/S2l/'

batch_size=32
memory_batch_size_train=266
memory_batch_size_test=170
next_batch_start=0
sample_batch_size=10

####  Loading model 

In [ ]:
## Defining placeholders in tf for images and targets
x_image = tf.placeholder(tf.float32, [None, 16,height,width,channel],name='x') 
y_true = tf.placeholder(tf.float32, [None, nb_classes],name='y_true')
y_true_cls = tf.placeholder(tf.int64, [None],name='y_true_cls')

model_keras = md.C3D_ucf101_training_model_tf(summary=False)
out=model_keras(x_image)
y_pred = tf.nn.softmax(out)
y_pred_cls = tf.argmax(out, dimension=1)
correct_prediction = tf.equal(y_pred_cls, y_true_cls)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print('Miscellenious items finished..!!',flush=True)

In [3]:
## Start the session with logging placement.---- I
init_op = tf.global_variables_initializer()
sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=False))
sess.run(init_op)

## Restore model weights from previously saved model
#saver = tf.train.Saver()
saver = tf.train.import_meta_graph(os.path.join(saved_path,'activity_model.ckpt-67.meta'))
saver.restore(sess, os.path.join(saved_path,'activity_model.ckpt-67'))
print("Model restored from file: %s" % saved_path,flush=True)

INFO:tensorflow:Restoring parameters from /nobackup/leopauly/S2l/activity_model.ckpt-67
Model restored from file: /nobackup/leopauly/S2l/


In [4]:
for op in sess.graph.get_operations():
    print(op.name)

init
Placeholder
Placeholder_1
Placeholder_2
Const
split/split_dim
split
Const_1
split_1/split_dim
split_1
conv1_input
conv1/random_uniform/shape
conv1/random_uniform/min
conv1/random_uniform/max
conv1/random_uniform/RandomUniform
conv1/random_uniform/sub
conv1/random_uniform/mul
conv1/random_uniform
conv1/kernel
conv1/kernel/Assign
conv1/kernel/read
conv1/Const
conv1/bias
conv1/bias/Assign
conv1/bias/read
conv1/convolution/dilation_rate
conv1/convolution
conv1/Reshape/shape
conv1/Reshape
conv1/add
conv1/Relu
pool1/MaxPool3D
conv2/random_uniform/shape
conv2/random_uniform/min
conv2/random_uniform/max
conv2/random_uniform/RandomUniform
conv2/random_uniform/sub
conv2/random_uniform/mul
conv2/random_uniform
conv2/kernel
conv2/kernel/Assign
conv2/kernel/read
conv2/Const
conv2/bias
conv2/bias/Assign
conv2/bias/read
conv2/convolution/dilation_rate
conv2/convolution
conv2/Reshape/shape
conv2/Reshape
conv2/add
conv2/Relu
pool2/MaxPool3D
conv3a/random_uniform/shape
conv3a/random_uniform/min
con

gradients/sequential_3/dropout_5/cond/Merge_grad/tuple/control_dependency
gradients/sequential_3/dropout_5/cond/Merge_grad/tuple/control_dependency_1
gradients/sequential_4/dropout_7/cond/Merge_grad/cond_grad
gradients/sequential_4/dropout_7/cond/Merge_grad/tuple/group_deps
gradients/sequential_4/dropout_7/cond/Merge_grad/tuple/control_dependency
gradients/sequential_4/dropout_7/cond/Merge_grad/tuple/control_dependency_1
gradients/Switch_8
gradients/Shape_9
gradients/zeros_8/Const
gradients/zeros_8
gradients/sequential_1/dropout_1/cond/Switch_1_grad/cond_grad
gradients/sequential_1/dropout_1/cond/dropout/mul_grad/Shape
gradients/sequential_1/dropout_1/cond/dropout/mul_grad/Shape_1
gradients/sequential_1/dropout_1/cond/dropout/mul_grad/BroadcastGradientArgs
gradients/sequential_1/dropout_1/cond/dropout/mul_grad/mul
gradients/sequential_1/dropout_1/cond/dropout/mul_grad/Sum
gradients/sequential_1/dropout_1/cond/dropout/mul_grad/Reshape
gradients/sequential_1/dropout_1/cond/dropout/mul_gr

gradients/sequential_3/dropout_5/cond/mul_grad/Reshape
gradients/sequential_3/dropout_5/cond/mul_grad/mul_1
gradients/sequential_3/dropout_5/cond/mul_grad/Sum_1
gradients/sequential_3/dropout_5/cond/mul_grad/Reshape_1
gradients/sequential_3/dropout_5/cond/mul_grad/tuple/group_deps
gradients/sequential_3/dropout_5/cond/mul_grad/tuple/control_dependency
gradients/sequential_3/dropout_5/cond/mul_grad/tuple/control_dependency_1
gradients/sequential_4/dropout_7/cond/mul_grad/Shape
gradients/sequential_4/dropout_7/cond/mul_grad/Shape_1
gradients/sequential_4/dropout_7/cond/mul_grad/BroadcastGradientArgs
gradients/sequential_4/dropout_7/cond/mul_grad/mul
gradients/sequential_4/dropout_7/cond/mul_grad/Sum
gradients/sequential_4/dropout_7/cond/mul_grad/Reshape
gradients/sequential_4/dropout_7/cond/mul_grad/mul_1
gradients/sequential_4/dropout_7/cond/mul_grad/Sum_1
gradients/sequential_4/dropout_7/cond/mul_grad/Reshape_1
gradients/sequential_4/dropout_7/cond/mul_grad/tuple/group_deps
gradients/s

####  Feature extraction

In [5]:
## Extraction of features 
def extract_video_features(vid):
    vid_=vid.reshape(-1,cluster_length,height,width,channel)
    f_v = sess.graph.get_tensor_by_name(('flatten_2/Reshape:0')) 
    #f_v_val=sess.run([f_v], feed_dict={'conv1_input:0':vid_,x_image:vid_,K.learning_phase(): 0 }) 
    f_v_val=np.array(sess.run([f_v], feed_dict={'conv1_input:0':vid_,'Placeholder:0':vid_ }))
    features=np.reshape(f_v_val,(-1))
    return features

In [6]:
## Uniform frame smapling
def get_compress_frames_data(filename, num_frames_per_clip=16):
  ''' Given a directory containing extracted frames, return a video clip of
  (num_frames_per_clip) consecutive frames as a list of np arrays '''
  ret_arr = []
  for parent, dirnames, filenames in os.walk(filename):
    filenames = sorted(filenames)
    jump=math.floor((len(filenames)/num_frames_per_clip))
    loop=0
    for i in range(0,len(filenames),jump):
      if (loop>15):
        break
      if (filenames[i].endswith('.png')):
        image_name = str(filename) + '/' + str(filenames[i])
        img = Image.open(image_name)
        img_data = np.array(img)
        ret_arr.append(img_data)
        loop=loop+1
  ret_arr=np.array(ret_arr)
  #ret_arr=ret_arr/255
  return np.array(ret_arr)

In [7]:
## Feature extraction

def get_features_from_class(class_folder):
    feature_set_a=[]
    base_dir_a=class_folder
    sub_dir_a=os.listdir(base_dir_a)
    sub_dir_a=sorted(sub_dir_a)
    if '.DS_Store' in sub_dir_a:
        sub_dir_a.remove('.DS_Store')
    print('\nSubdirectories:',sub_dir_a)
    for sub_dir_a_ in sub_dir_a:
        vid_a=get_compress_frames_data(base_dir_a+sub_dir_a_)
        feature_set_a.append(extract_video_features(vid_a))
    return np.array(feature_set_a)

In [8]:
feature_set_a=get_features_from_class('/nobackup/leopauly/S2l/Dataset/Leeds_Dataset_ Only_tasks_CompvsIncomp/Push/')


Subdirectories: ['push_demo_0deg', 'push_demo_180deg', 'push_demo_human', 'push_robo', 'push_robo_M2', 'push_robo_M3', 'push_robo_arbview1', 'push_robo_bg_fast', 'push_robo_change_pos', 'push_robo_changetarget', 'push_robo_fast', 'push_robo_green', 'push_robo_obj2_new', 'push_robo_sideview']


InvalidArgumentError: Shape [-1,16,112,112,3] has negative dimensions
	 [[Node: conv1_input_1 = Placeholder[dtype=DT_FLOAT, shape=[?,16,112,112,3], _device="/job:localhost/replica:0/task:0/gpu:0"]()]]

Caused by op 'conv1_input_1', defined at:
  File "/opt/conda/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/opt/conda/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/opt/conda/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/opt/conda/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/opt/conda/lib/python3.6/site-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
  File "/opt/conda/lib/python3.6/site-packages/tornado/i

In [ ]:
feature_set_b=get_features_from_class('/nobackup/leopauly/S2l/Dataset/Leeds_Dataset_ Only_tasks_CompvsIncomp/Reach/')

In [ ]:
feature_set_c=get_features_from_class('/nobackup/leopauly/S2l/Dataset/Leeds_Dataset_ Only_tasks_CompvsIncomp/Reach n Push/')

#### Clustering 

In [ ]:
kmeans = KMeans(n_clusters=3,random_state=1)
points=np.concatenate((feature_set_a,feature_set_b,feature_set_c),axis=0)
kmeans.fit(points)
y_km = kmeans.fit_predict(points)

In [ ]:
print(y_km[0:14])
print(y_km[14:28])
print(y_km[28:42])

In [ ]:
## Accuracy for our method
y_true=[0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2]
print('Adjusted random score:',round(metrics.adjusted_rand_score(y_true,y_km),2))
print('Adjusted mutual infromation score:',round(metrics.adjusted_mutual_info_score(y_true,y_km),2))
print('Homogeneity score:',round(metrics.homogeneity_score(y_true,y_km),2))

In [ ]:
sess.close()